# RNN

## Prelims


In [ ]:
import math, time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from keras.datasets import imdb

from rnn import RNN

<div>
<img src="https://stanford.edu/~shervine/teaching/cs-230/illustrations/architecture-rnn-ltr.png?9ea4417fc145b9346a3e288801dbdfdc" width="800"/>
</div>

# TESTING

In [2]:
from rnn import RNN
import numpy as np

vocab_size = 5

###
seq1 = np.zeros((3,vocab_size,1))
s1 = [0,2,4]

for i,val in enumerate(s1):
    seq1[i][val] = 1

y1 = np.zeros((2,1))
y1[0] = 1


###
seq2 = np.zeros((4,vocab_size,1))
s2 = [1,3,2,0]

for i,val in enumerate(s2):
    seq2[i][val] = 1

y2 = np.zeros((2,1))
y2[1] = 1

###
seq3 = np.zeros((3,vocab_size,1))
s3 = [0,3,1]

for i,val in enumerate(s3):
    seq3[i][val] = 1

y3 = np.zeros((2,1))
y3[0] = 1

Xtrain = [seq1, seq2]
Ytrain = [y1, y2]

Xtest = [seq3]
Ytest = [y3]

rnntest = RNN(input_dim=vocab_size, output_dim=2, hidden_dim=5, learning_rate=.01)
rnntest.fit(Xtrain, Ytrain, num_epochs=20)

rnntest.evaluate(Xtest, Ytest)

Epoch 1, training loss: 1.4667165699747788
Epoch 2, training loss: 1.455980251851398
Epoch 3, training loss: 1.4452723907056724
Epoch 4, training loss: 1.434454872584022
Epoch 5, training loss: 1.423383039124447
Epoch 6, training loss: 1.4119031180497468
Epoch 7, training loss: 1.3998511155225049
Epoch 8, training loss: 1.387065534782919
Epoch 9, training loss: 1.3733692842295198
Epoch 10, training loss: 1.358423010590644
Epoch 11, training loss: 1.3423450367674967
Epoch 12, training loss: 1.325082799335659
Epoch 13, training loss: 1.3064898650505519
Epoch 14, training loss: 1.2864152210408415
Epoch 15, training loss: 1.26468831436911
Epoch 16, training loss: 1.2412999807794038
Epoch 17, training loss: 1.216207276966697
Epoch 18, training loss: 1.1893176615094916
Epoch 19, training loss: 1.160614442306576
Epoch 20, training loss: 1.1302091653125252


0.0

## Evaluation on IMDB Dataset

Words are indexed by overall frequency in the dataset e.g. `3` encodes the third most frequent word.


In [46]:
from keras.datasets import imdb
import numpy as np
vocab_size = 100

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

word_index = imdb.get_word_index()
index_to_word = dict([(value,key) for (key,value) in word_index.items()])

(X_train_small, y_train_small) = (X_train[0:100], y_train[0:100])
(X_test_small, y_test_small) = (X_test[0:100], y_test[0:100])

In [47]:
def decode(input):
    sentence = []
    for i in input:
        sentence.append(index_to_word.get(i-3, '!'))
    print(" ".join(sentence))
    return sentence

def one_hot(value, vec_size):
    """
    Given a scalar returns the one-hot encoding vector.
    """
    # initialize input matrix
    x = np.zeros((vec_size, 1))
    x[value] = 1
        
    return x

def subtract_one(X):
    """
    Subtracts one from each value.
    """
    for i in range(X.size):
        X[i] = [x-1 for x in X[i]]
    
    return X

def one_hot_x(X):
    oh = [None] * X.size
    for i, input in enumerate(X):
        xx = np.zeros((len(input), vocab_size, 1))
        for j, val in enumerate(input):
            xx[j][val] = 1
        oh[i] = xx
    
    return oh

def one_hot_y(Y):
    oh = [None] * Y.size
    for i, y in enumerate(Y):
        oh[i] = one_hot(y, 2)
    
    return oh

In [48]:
# subtract ones
X_train_small = subtract_one(X_train_small)
X_test_small = subtract_one(X_test_small)  

In [49]:
tx = one_hot_x(X_train_small)
ty = one_hot_y(y_train_small)
testingx = one_hot_x(X_test_small)
testingy = one_hot_y(y_test_small)

In [51]:
from rnn import RNN
RNNModel = RNN(input_dim=vocab_size, hidden_dim=10, output_dim=2, learning_rate=1e-4)
RNNModel.fit(tx, ty, 10)

Epoch 1, training loss: 38.02021500786034
Epoch 2, training loss: 37.88570052221522
Epoch 3, training loss: 37.76739698482241
Epoch 4, training loss: 37.66479923610425
Epoch 5, training loss: 37.57031353036847
Epoch 6, training loss: 37.48792986875993
Epoch 7, training loss: 37.406753961560895
Epoch 8, training loss: 37.32507970195589
Epoch 9, training loss: 37.24167319569322
Epoch 10, training loss: 37.158977623183155


[38.02021500786034,
 37.88570052221522,
 37.76739698482241,
 37.66479923610425,
 37.57031353036847,
 37.48792986875993,
 37.406753961560895,
 37.32507970195589,
 37.24167319569322,
 37.158977623183155]

In [45]:
RNNModel.evaluate(testingx, testingy)
# RNNModel.evaluate(tx, ty)

0.43